In [3]:
%load_ext autoreload
%autoreload 2
import control
import napari

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
config = {
    "camera": {
        "device": "zyla_camera"
    },
    "objective": {
        "device": "ti_objective",
        "states": ["a", "b", "c", "d", "e", "f"]
    },
    "light": {
        "device": "sola_light",
        "port": "COM4"
    },
    "up_filter": {
        "device": "ti_filter1"
    },
    "down_filter": {
        "device": "ti_filter1"
    },
    "stage": {
        "device": "asi_stage",
        "port": "COM5"
    },
    "focus": {
        "device": "ti_focus"
    },
    "lightpath": {
        "device": "ti_lightpath"
    },
    "uv_filter": {
        "device": "lambda_filter1",
        "port": "COM3"
    },
    "right_filter": {
        "device": "lambda_filter2",
        "port": "COM3"
    },
    "laser": {
        "device": "lambda_shutter1",
        "port": "COM3"
    },
    "uv_shutter": {
        "device": "lambda_shutter2",
        "port": "COM3"
    },
}
c = control.load(config)